In [4]:
import numpy as np
import pandas as pd

locations = pd.read_csv("FinalProject - Costco.txt")

locations.head()

,1290 US HIGHWAY 22E,North Plainfield,NJ 07060
0,9700 E Kellogg Ave,Wichita,KS 67207
1,200 Costco Way,St Peters,MO 63376
2,202 Costco Dr,Pittsburgh,PA 15205
3,10220 S Memorial Dr,Tulsa,OK 74133
4,12221 Blue Valley Pkwy,Overland Park,KS 66213


In [5]:
import requests # library to handle requests

CLIENT_ID = 'BRBCOMIZ1Y12L11GXPQYR2CVT4PX20F4GSYYXKYD4HT4DTPF' # your Foursquare ID
CLIENT_SECRET = 'AJR5VTLYGN5ZNX05DFNA2EH2IX1DF0DAPOFWILQR2IX2KDWL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 20

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BRBCOMIZ1Y12L11GXPQYR2CVT4PX20F4GSYYXKYD4HT4DTPF
CLIENT_SECRET:AJR5VTLYGN5ZNX05DFNA2EH2IX1DF0DAPOFWILQR2IX2KDWL


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()